# Formation DSA - Fabien FAIVRE

**TD Antoine LY : analyse de sentiments**

L'objectif est de vous faire approfondir une notion de machine learning aux travers d'une compétition Kaggle. Kaggle est aujourd'hui un site de compétition incontournable dans le monde du machine learning. Même si la plateforme n'est pas représentative des enjeux opérationnels, elle est néanmoins représentative des sujets d'attention de la communauté scientifique et demeure un bon outil d'apprentissage est de partage.


Le projet consiste donc à utiliser le challenge [Tweet sentiment extraction](https://www.kaggle.com/c/tweet-sentiment-extraction/overview/description) à des fins académiques.


### Les données

Les données sont celles proposées par le challenge. Elles composent de deux fichiers:

* `train.csv` ce document comportent les données à utiliser pour calibrer votre modèle. Il comporte toutes les colonnes
* `test.csv` ce document n'est utilisé **que** pour évaluer la performance finale de votre modèle. En aucun cas il ne peut être utilisé pour fine-tuner ou calibrer votre modèle. Il simule les données qui ne sont normalement JAMAIS accessible sur Kaggle (ni dans la vraie vie). à considérer comme un nouvel échantillon.

### Le challenge

Prédire la colonne `sentiment` à partir de la colonne `text`.

### La métrique d'évaluation

On utilisera un score F1 à l'aide de la fonction implémentée dans `scikit-learn`

    from sklearn.metrics import f1_score
    y_true = [0, 1, 2, 0, 1, 2]
    y_pred = [0, 2, 1, 0, 0, 1]
    f1_score(y_true, y_pred, average='macro')


### Labels à utiliser pour la colonne `sentiment`

Vous devrez retraiter la colonne `sentiment` en utilisant les remplacements suivants:

    "neutral"  ->  0
    "negative" -> -1
    "positive" ->  1

### Informations pratiques sur le rendu et la notation.

L'objectif est de se familiariser avec les techniques de text-mining à des fins de classification de sentiments d'un texte. La notation se décomposera en deux parties:

#### Notation

* Votre méthodologie et votre approche (12 points) : cette partie doit mettre en avant la motivation des différents retraitements que vous avez appliquez, votre effort de comprendre les implémentations des pacakges que vous aurez utilisés ainsi que le bon sens que leurs utilisations transcrit.
* La performance finale et méthodologie (4 + 4 = 8 points) : cette notation sera relative au groupe. 3 tentatives d'algorithmes/preprocessing différents permettrons de garantir 4 points sur les 8. Les 3 premiers du classement (du groupe 2020) calculé à l'aide du score F1 sur la base de test atteigneront 4 points supplémentaires. Le reste du barême relatif au classement sera dégressif de façon linéaire par palier: les derniers obtenant 1 point minimum.

#### Rendu

Le rendu se fera sous la forme d'un court rapport (max 5 pages). Ce dernier peut se faire sous la forme d'un notebook (html ou pdf) ou d'un rapport traditionnel (word, pdf). Il doit mettre en avant la méthodologie employée, les difficultés rencontrées ainsi que les différents apprentissages.


Le projet sera à rendre lors de la séance de **Juillet 2021** de restitution.

### Language de programmation

Il est fortement recommandé d'effectuer le projet en python, mais ceci n'est pas obligatoire.

Bibliographie:

https://www.scor.com/fr/articles-experts/accroitre-vitesse-et-precision-grace-lexploration-de-texte-et-au-traitement

# Approche méthodologique

L'approche suivie est double. Elle a consisté à :
- prendre en main les approches de traitement du NLP en suivant la gradation historique (Wordvectors => TF-IdF => Word2Vec => DL avec notamment l'implémentation ` Twitter-roBERTa-base for Sentiment Analysis` sur [HuggingFace](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment).
- s'approprier des techniques de MLOps au travers de l'utilisation et de l'adaptation d'un instanciation particulière de cookiecutter par [Manifold.ai](https://github.com/manifoldai/orbyter-cookiecutter). Ce template permet d'instancier deux environnement dockerisés (l'un pour l'environnement de développement et l'autre pour un serveur ML_Flow). L'ensemble des travaux a été versionné dans ce [repo github](https://github.com/Fabien-DS/DSA_Sentiment). L'objectif de cette difficulté compélmentaire était de tester en situation réel un ensemble d'outillages potentiellement intéressants pour MACIF.

## 0) Création des prérequis

Cette section sert à mettre en place l'environnement de travail
- charger les packages nécessaires au projet
- permettre de refactoriser le code du projet sous forme de package
- mettre un place un tracking d'experiment avec ML_Flow pour suivre les itérations du projet

### Import des packages utilisés

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px

import os

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
import re


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC

from xgboost import XGBClassifier


import spacy 

from sklearn.metrics import f1_score

import mlflow
import mlflow.sklearn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Utilisation du code du projet packagé

In [2]:
#Cette cellule permet d'appeler la version packagée du projet et d'en assurer le reload avant appel des fonctions
%load_ext autoreload
%autoreload 2

### Configuration de l'experiment MLFlow

In [4]:
mlflow.tracking.get_tracking_uri()

'/mnt/experiments'

In [5]:
exp_name="DSA_sentiment"
mlflow.set_experiment(exp_name)

INFO: 'DSA_sentiment' does not exist. Creating a new experiment


### Chargement des données

In [31]:
data_folder = os.path.join('..', 'data', 'raw')
all_raw_files = [os.path.join(data_folder, fname)
                    for fname in os.listdir(data_folder)]
all_raw_files

['../data/raw/sample_submission.csv',
 '../data/raw/test.csv',
 '../data/raw/train.csv']

In [32]:
documents = {}
for text_fname in all_raw_files:
    bname = os.path.basename(text_fname).split('.')[0]
    documents[bname] = pd.read_csv(text_fname, encoding='utf8')

In [33]:
train = pd.DataFrame(documents['train'])
test = pd.DataFrame(documents['test'])

In [34]:
train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [73]:
train.tail()

,textID,text,selected_text,sentiment
27475,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27476,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27477,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27478,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive
27479,6f7127d9d7,All this flirting going on - The ATG smiles...,All this flirting going on - The ATG smiles. Y...,neutral


In [35]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3534 entries, 0 to 3533
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   textID     3534 non-null   object
 1   text       3534 non-null   object
 2   sentiment  3534 non-null   object
dtypes: object(3)
memory usage: 83.0+ KB


In [36]:
print(
    'train : \n',
    train.isna().sum(), 
    '\n',
    'test : \n',
    test.isna().sum()
)


train : 
 textID           0
text             1
selected_text    1
sentiment        0
dtype: int64 
 test : 
 textID       0
text         0
sentiment    0
dtype: int64


Il n'est pas possible de faire de l'imputation comme avec des champs numérique. Il convient donc de supprimer les entrées vides

In [37]:
train.dropna(inplace=True)
train = train.reset_index(drop=True)
test.dropna(inplace=True)
test = test.reset_index(drop=True)


## Model

In [38]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.field]

class NumberSelector(BaseEstimator, TransformerMixin):
    def __init__(self, field):
        self.field = field
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.field]]

In [39]:
def Tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    porter_stemmer=nltk.PorterStemmer()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [40]:
def transform_target(st):
    st =  -1 if st=="negative" else 1 if st=="positive" else 0
    return st

In [41]:
classifier = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('text')), #Sélection de la colonne à transformer (corpus)
            ('tfidf', TfidfVectorizer(tokenizer=Tokenizer, stop_words=stopwords.words('english'), #Sélection de la colonne à transformer (corpus)
                     min_df=.0025, max_df=0.25, ngram_range=(1,3))),
            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)), #for XGB : linear dimensionality reduction by means of truncated singular value decomposition (SVD)
        ]))
    ])),
    ('clf', XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.1)),
#    ('clf', RandomForestClassifier()),
    ])

In [42]:
X_train = pd.DataFrame(train['text'].apply(lambda x : str(x).lower()), columns=['text'])
y_train=train['sentiment'].apply(lambda x : transform_target(x))
X_test = pd.DataFrame(test['text'].apply(lambda x : str(x).lower()), columns=['text'])
y_test=test['sentiment'].apply(lambda x : transform_target(x))

In [43]:
y_train.head()

0    0
1   -1
2   -1
3   -1
4   -1
Name: sentiment, dtype: int64

In [44]:
X_train.head()

,text
0,"i`d have responded, if i were going"
1,sooo sad i will miss you here in san diego!!!
2,my boss is bullying me...
3,what interview! leave me alone
4,"sons of ****, why couldn`t they put them on t..."


In [45]:
classifier.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '
/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:42:19] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text',
                                                 Pipeline(steps=[('colext',
                                                                  TextSelector(field='text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer(max_df=0.25,
                                                                                  min_df=0.0025,
                                                                                  ngram_range=(1,
                                                                                               3),
                                                                                  stop_words=['i',
                                                                                              'me',
                                                                                

In [46]:
y_test_pred = classifier.predict(X_test)

In [47]:
f1_score(y_test, y_test_pred, average='macro')

0.6569194215147925

In [48]:
classifier2 = Pipeline([
    ('features', FeatureUnion([
        ('text', Pipeline([
            ('colext', TextSelector('text')), #Sélection de la colonne à transformer (corpus)
            ('tfidf', TfidfVectorizer())
        ]))
    ])),
    ('clf', LinearSVC()),
#    ('clf', RandomForestClassifier()),
    ])

In [49]:
classifier2.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text',
                                                 Pipeline(steps=[('colext',
                                                                  TextSelector(field='text')),
                                                                 ('tfidf',
                                                                  TfidfVectorizer())]))])),
                ('clf', LinearSVC())])

In [50]:
y_test_pred2 = classifier2.predict(X_test)

In [51]:
f1_score(y_test, y_test_pred2, average='macro')

0.6873879020805563

In [52]:
bow_pipeline = Pipeline(
    steps=[
        ('colext', TextSelector('text')), #Sélection de la colonne à transformer (corpus)
        ("tfidf", TfidfVectorizer()),
        ("classifier", RandomForestClassifier(n_jobs=-1)),
    ]
)
bow_pipeline.fit(X_train, y_train)
y_pred = bow_pipeline.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6902256127441538

In [69]:
import spacy

In [72]:
nlp = spacy.load("en_core_web_md")  # this model will give you 300D
#nlp = spacy.load("en_core_web_trf")

In [56]:
class SpacyVectorTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, nlp):
        self.nlp = nlp
        self.dim = 300

    def fit(self, X, y):
        return self

    def transform(self, X):
        # Doc.vector defaults to an average of the token vectors.
        # https://spacy.io/api/doc#vector
        return [self.nlp(text).vector for text in X]

In [60]:
embeddings_pipeline = Pipeline(
    steps=[
        ('colext', TextSelector('text')), #Sélection de la colonne à transformer (corpus)
        ("mean_embeddings", SpacyVectorTransformer(nlp)),
        ("reduce_dim", TruncatedSVD(50)),
        ("classifier", RandomForestClassifier(n_jobs=-1)),
    ]
)
embeddings_pipeline.fit(X_train, y_train)
y_pred = embeddings_pipeline.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6043214555569574

In [61]:
embeddings_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'colext', 'mean_embeddings', 'reduce_dim', 'classifier', 'colext__field', 'mean_embeddings__nlp', 'reduce_dim__algorithm', 'reduce_dim__n_components', 'reduce_dim__n_iter', 'reduce_dim__random_state', 'reduce_dim__tol', 'classifier__bootstrap', 'classifier__ccp_alpha', 'classifier__class_weight', 'classifier__criterion', 'classifier__max_depth', 'classifier__max_features', 'classifier__max_leaf_nodes', 'classifier__max_samples', 'classifier__min_impurity_decrease', 'classifier__min_impurity_split', 'classifier__min_samples_leaf', 'classifier__min_samples_split', 'classifier__min_weight_fraction_leaf', 'classifier__n_estimators', 'classifier__n_jobs', 'classifier__oob_score', 'classifier__random_state', 'classifier__verbose', 'classifier__warm_start'])

In [62]:
bow_pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'colext', 'tfidf', 'classifier', 'colext__field', 'tfidf__analyzer', 'tfidf__binary', 'tfidf__decode_error', 'tfidf__dtype', 'tfidf__encoding', 'tfidf__input', 'tfidf__lowercase', 'tfidf__max_df', 'tfidf__max_features', 'tfidf__min_df', 'tfidf__ngram_range', 'tfidf__norm', 'tfidf__preprocessor', 'tfidf__smooth_idf', 'tfidf__stop_words', 'tfidf__strip_accents', 'tfidf__sublinear_tf', 'tfidf__token_pattern', 'tfidf__tokenizer', 'tfidf__use_idf', 'tfidf__vocabulary', 'classifier__bootstrap', 'classifier__ccp_alpha', 'classifier__class_weight', 'classifier__criterion', 'classifier__max_depth', 'classifier__max_features', 'classifier__max_leaf_nodes', 'classifier__max_samples', 'classifier__min_impurity_decrease', 'classifier__min_impurity_split', 'classifier__min_samples_leaf', 'classifier__min_samples_split', 'classifier__min_weight_fraction_leaf', 'classifier__n_estimators', 'classifier__n_jobs', 'classifier__oob_score', 'classifier__random_state'

In [63]:
from sklearn.model_selection import RandomizedSearchCV# the keys can be accessed with final_pipeline.get_params().keys()
params = {
    "tfidf__use_idf": [True, False],
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "classifier__bootstrap": [True, False],
    "classifier__class_weight": ["balanced", None],
    "classifier__n_estimators": [100, 300, 500, 800, 1200],
    "classifier__max_depth": [5, 8, 15, 25, 30],
    "classifier__min_samples_split": [2, 5, 10, 15, 100],
    "classifier__min_samples_leaf": [1, 2, 5, 10]
}

search = RandomizedSearchCV(bow_pipeline, params)
search.fit(X_train, y_train)
y_pred = search.predict(X_test)
f1_score(y_test, y_pred, average='macro')

0.6969276370587574

In [65]:
search.best_params_

{'tfidf__use_idf': False,
 'tfidf__ngram_range': (1, 1),
 'classifier__n_estimators': 1200,
 'classifier__min_samples_split': 10,
 'classifier__min_samples_leaf': 2,
 'classifier__max_depth': 25,
 'classifier__class_weight': 'balanced',
 'classifier__bootstrap': True}

In [74]:
from sklearn.metrics import classification_report

In [76]:
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

          -1       0.67      0.69      0.68      1001
           0       0.66      0.68      0.67      1430
           1       0.76      0.72      0.74      1103

    accuracy                           0.69      3534
   macro avg       0.70      0.70      0.70      3534
weighted avg       0.70      0.69      0.69      3534

